## Concatenating attention distributions 6, 7, and 8 using a GRU 

We extract the attention distribution directly from a model that has been pretrained on bias detection!

We concatenate together the 6th, 7th, and 8th attention distributions of the input. Our classification model is an LSTM that reads in the attention distribution for each word sequentially. In general the max sequence lenght is 80 which means that our model will read in 80 data points which are each 80 dimensional. 


#### Notes
* One remaining question is how can we experiment with different number of attention heads. In general the extract attention scores function seems to have some bugs that need ironing out - such as only being able to pass in a batch size of 1 into the extraction schema.

In [1]:
import sys; sys.path.append("../../../../..")
import torch 
from src.experiment import AttentionExperiment, ClassificationExperiment
from src.dataset import ExperimentDataset
from src.params import Params
from src.utils.attention_utils import reduce_attention_dist
from src.utils.classification_utils import run_bootstrapping

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
params = Params.read_params("experiment_params.json")
print("layers = {}".format(params.intermediary_task["attention"]["layers"]))
print("reducer = {}".format(params.intermediary_task["attention"]["reducer"]))

layers = [6, 7, 8]
reducer = concat


In [ ]:
# Loading in the dataset that we are using in this experiments 
# typically this dataset is the small set of ground-truth labels
dataset = ExperimentDataset.init_dataset(params.dataset)
dataset

Attention Experiment: 
* Is a class that wraps useful methods to extract attention distributions from a given BERT-based model 
* The user has to provide in two config files: One to specify parameters for how the attention scores should be extracted and combined, and other to specify the intermediary model from which the attention scores should be extracted from
* The user needs to instantiate the attention experiment with a function that tells the model how to run 
 inference on the given model. The function header is specified below: 
 
 ``` def initialize_attention_experiment(cls, intermediary_task_params, dataset_params, verbose=False) ```
 


In [ ]:
attention_dataloader = dataset.return_dataloader() 
attention_experiment = AttentionExperiment.initialize_attention_experiment(params.intermediary_task, params.dataset, verbose=True)

```extract_attention_scores()``` works out of the box because the attention experiment has the config file saved, and knows what BERT model to use/load in, which layers to extract the attention scores from, and what the inference function is that should be used on this particular BERT model.

Attention_scores is then a list of dictionaries. The keys in this dictionary are the specific layers of a BERT model and the values are the corresponding attention distributions extracted from that particular layer.

In [6]:
attention_scores = attention_experiment.extract_attention_scores(attention_dataloader)

In [7]:
reduced_attention = reduce_attention_dist(attention_scores, params.intermediary_task["attention"]["reducer"])
stacked_reduced_attention = torch.stack(reduced_attention).squeeze()
dataset.add_data(stacked_reduced_attention, "attention_dist")
dataset.shuffle_data()
print("reduced_attention[0].shape = {}".format(reduced_attention[0].shape))
print("stacked_reduced_attention.shape = {}".format(stacked_reduced_attention.shape))
dataset

reduced_attention[0].shape = torch.Size([1, 1, 80, 240])
stacked_reduced_attention.shape = torch.Size([324, 80, 240])


Length: 324 Keys: dict_keys(['pre_ids', 'masks', 'pre_lens', 'post_in_ids', 'post_out_ids', 'pre_tok_label_ids', 'post_tok_label_ids', 'rel_ids', 'pos_ids', 'categories', 'index', 'bias_label', 'attention_dist'])

### This is where the classification experiment starts

We create a classification experiment that contains useful methods for classifying bias based on the attention distributions. 

In [8]:
classification_experiment = ClassificationExperiment.init_cls_experiment(params.final_task)

In [9]:
stats = run_bootstrapping(classification_experiment, dataset, params.final_task, num_bootstrap_iters=5, input_key="attention_dist", label_key="bias_label")

In [10]:
stats

{'auc': [(0.5041903725052774, 0.6195650759629088), 0.5801859720210008],
 'accuracy': [(0.594579081632653, 0.69375), 0.6344812925170069]}

In [11]:
classification_experiment.save_model_weights("gru-attention.weights")